In [99]:
import os
import pandas as pd

In [100]:
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
cln_pkl_loc = os.path.join(ROOT_DIR, 'data_cleaning','cleanweathersmall.pkl')

In [101]:
df = pd.read_pickle(cln_pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3765246 entries, 0 to 3765245
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   station  object        
 1   time     datetime64[ns]
 2   temp     float64       
 3   dwpt     float64       
 4   rhum     float64       
 5   prcp     float64       
 6   wdir     float64       
 7   wspd     float64       
 8   pres     float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 258.5+ MB


In [102]:
pivoted_df = df.pivot(index='time', columns='station', values=['temp', 'dwpt','rhum','prcp','wdir','wspd','pres'])
pivoted_df.columns = ['_'.join(col) for col in pivoted_df.columns.values]
pivoted_df

,temp_0CNUO,temp_0CO7B,temp_0FV1F,temp_1J1PJ,temp_1JWST,temp_20QWH,temp_21O3U,temp_2W8UZ,temp_3S56J,temp_4DUJO,...,pres_W5F5F,pres_X9FED,pres_XM44W,pres_Y59TE,pres_Z7ZOG,pres_ZFZUV,pres_ZNWZW,pres_ZUQJS,pres_ZWC6W,pres_ZYITU
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,-25.1,5.0,1.2,-0.9,-6.5,-28.7,-29.0,4.6,-26.5,1.8,...,NaN,NaN,1013.5,NaN,NaN,1023.8,NaN,NaN,1010.8,NaN
2022-01-01 01:00:00,-25.7,5.1,0.9,-0.6,-7.5,-29.5,-28.9,4.6,-27.3,1.6,...,NaN,NaN,1014.6,NaN,NaN,1024.2,NaN,NaN,1011.8,NaN
2022-01-01 02:00:00,-26.2,4.2,1.5,-0.8,-8.6,-30.2,-30.1,4.7,-27.9,0.9,...,NaN,NaN,1015.1,NaN,NaN,1024.9,NaN,NaN,1012.7,NaN
2022-01-01 03:00:00,-26.1,5.5,4.1,-0.9,-9.4,-29.9,-30.1,4.6,-28.1,1.0,...,NaN,NaN,1016.0,NaN,NaN,1025.4,NaN,NaN,1013.6,NaN
2022-01-01 04:00:00,-26.5,5.9,3.1,-1.2,-10.1,-30.4,-31.4,4.9,-28.6,0.5,...,NaN,NaN,1017.3,NaN,NaN,1025.4,NaN,NaN,1013.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22 19:00:00,12.4,17.8,13.7,11.5,10.2,13.5,12.5,15.2,10.8,12.4,...,1022.1,1025.2,1022.6,1025.0,1025.3,1024.5,1025.4,1022.6,1017.8,1016.7
2022-09-22 20:00:00,13.0,17.0,14.3,12.0,10.4,14.0,12.5,14.8,11.4,11.6,...,1020.3,1024.4,1022.4,1024.0,1024.0,1023.1,1025.0,1022.2,1017.9,1016.0
2022-09-22 21:00:00,13.2,16.6,13.9,12.4,10.5,14.3,12.4,15.1,11.7,10.8,...,1019.6,1024.2,1022.2,1023.0,1023.7,1022.0,1024.0,1022.4,1018.1,1017.4


### Our target is Ann Arbor which is station KARB0. This simple model will just try to predict the temperature for KARB0. So will build some features specifically around it.

In [103]:
ann_arbor_cols = [col for col in pivoted_df.columns if "KARB0" in col]
ann_arbor_df = pivoted_df[ann_arbor_cols].copy()

#### This adds a columnn for the weather at 24-48 hrs prior to our target

In [104]:
for col in ann_arbor_df.columns:
    for i in range(-24,-3):
        ann_arbor_df[f'{i}hr~{col}'] = ann_arbor_df[col].shift(i)

C:\Users\jesse\AppData\Local\Temp\ipykernel_15776\2526676930.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ann_arbor_df[f'{i}hr~{col}'] = ann_arbor_df[col].shift(i)
C:\Users\jesse\AppData\Local\Temp\ipykernel_15776\2526676930.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ann_arbor_df[f'{i}hr~{col}'] = ann_arbor_df[col].shift(i)
C:\Users\jesse\AppData\Local\Temp\ipykernel_15776\2526676930.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

In [105]:
ann_arbor_df = ann_arbor_df.rename_axis(None, axis = 0)
ann_arbor_df

,temp_KARB0,dwpt_KARB0,rhum_KARB0,prcp_KARB0,wdir_KARB0,wspd_KARB0,pres_KARB0,-24hr~temp_KARB0,-23hr~temp_KARB0,-22hr~temp_KARB0,...,-13hr~pres_KARB0,-12hr~pres_KARB0,-11hr~pres_KARB0,-10hr~pres_KARB0,-9hr~pres_KARB0,-8hr~pres_KARB0,-7hr~pres_KARB0,-6hr~pres_KARB0,-5hr~pres_KARB0,-4hr~pres_KARB0
2022-01-01 00:00:00,5.0,2.9,86.0,0.0,120.0,11.0,1005.0,-2.2,-1.1,-0.6,...,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6,1005.3,1004.9
2022-01-01 01:00:00,5.0,2.9,86.0,0.0,120.0,11.2,1005.4,-2.8,-2.2,-1.1,...,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6,1005.3
2022-01-01 02:00:00,5.0,2.9,86.0,0.0,120.0,5.4,1005.3,-3.3,-2.8,-2.2,...,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6
2022-01-01 03:00:00,5.0,2.9,86.0,0.0,0.0,0.0,1004.9,-3.9,-3.3,-2.8,...,1009.6,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6
2022-01-01 04:00:00,5.6,3.3,85.0,0.0,0.0,0.0,1004.9,-4.4,-3.9,-3.3,...,1008.9,1009.6,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22 19:00:00,15.0,3.5,46.0,0.0,320.0,25.9,1017.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1019.0
2022-09-22 20:00:00,16.1,4.5,46.0,0.0,320.0,18.4,1017.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-22 21:00:00,16.0,4.4,46.0,0.0,330.0,22.0,1019.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-22 22:00:00,14.0,2.9,47.0,0.0,330.0,22.0,1019.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
pred_df = pd.merge(pivoted_df,ann_arbor_df, left_index=True, right_index=True)
pred_df

,temp_0CNUO,temp_0CO7B,temp_0FV1F,temp_1J1PJ,temp_1JWST,temp_20QWH,temp_21O3U,temp_2W8UZ,temp_3S56J,temp_4DUJO,...,-13hr~pres_KARB0,-12hr~pres_KARB0,-11hr~pres_KARB0,-10hr~pres_KARB0,-9hr~pres_KARB0,-8hr~pres_KARB0,-7hr~pres_KARB0,-6hr~pres_KARB0,-5hr~pres_KARB0,-4hr~pres_KARB0
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,-25.1,5.0,1.2,-0.9,-6.5,-28.7,-29.0,4.6,-26.5,1.8,...,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6,1005.3,1004.9
2022-01-01 01:00:00,-25.7,5.1,0.9,-0.6,-7.5,-29.5,-28.9,4.6,-27.3,1.6,...,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6,1005.3
2022-01-01 02:00:00,-26.2,4.2,1.5,-0.8,-8.6,-30.2,-30.1,4.7,-27.9,0.9,...,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6,1004.6
2022-01-01 03:00:00,-26.1,5.5,4.1,-0.9,-9.4,-29.9,-30.1,4.6,-28.1,1.0,...,1009.6,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7,1004.6
2022-01-01 04:00:00,-26.5,5.9,3.1,-1.2,-10.1,-30.4,-31.4,4.9,-28.6,0.5,...,1008.9,1009.6,1009.0,1008.5,1007.7,1007.0,1005.8,1005.4,1005.6,1005.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22 19:00:00,12.4,17.8,13.7,11.5,10.2,13.5,12.5,15.2,10.8,12.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1019.0
2022-09-22 20:00:00,13.0,17.0,14.3,12.0,10.4,14.0,12.5,14.8,11.4,11.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-22 21:00:00,13.2,16.6,13.9,12.4,10.5,14.3,12.4,15.1,11.7,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
pred_df = pred_df[pred_df['-24hr~temp_KARB0'].notna()]

In [108]:
to_drop = []
for col in pred_df.columns:
    num = pred_df[col].isna().sum()
    if num > 100:
        # print(f"{col} has {num} missing values")
        to_drop.append(col)
pred_df.drop(columns=to_drop,inplace=True)
pred_df.dropna(inplace=True)
pred_df.shape

C:\Users\jesse\AppData\Local\Temp\ipykernel_15776\1722294100.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df.drop(columns=to_drop,inplace=True)
C:\Users\jesse\AppData\Local\Temp\ipykernel_15776\1722294100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df.dropna(inplace=True)


(6298, 4164)

### Now our target will be the '-24hr~temp_KARB0' column, which is the temperature 24 hrs in the future from all of the original features.

In [116]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

X_cols = [col for col in pred_df.columns if "~" not in col]
X = pred_df[X_cols]
y = pred_df['-24hr~temp_KARB0']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
reg = DecisionTreeRegressor(random_state=0)
cross_val_score(reg, X, y, cv=10)

array([-0.34531993,  0.24644689,  0.39111481,  0.00903564,  0.64512049,
        0.36926452,  0.52377484,  0.4560356 ,  0.47348592,  0.31957405])

In [117]:
from sklearn.ensemble import GradientBoostingRegressor
gbreg = GradientBoostingRegressor(random_state=0)
cross_val_score(gbreg, X, y, cv=10)


array([0.49297321, 0.46960416, 0.7281656 , 0.63523179, 0.79612935,
       0.67508297, 0.74230197, 0.76748594, 0.72720562, 0.68103795])